In [1]:
from stormvogel import visualization, map, result
import stormpy

In [2]:
%%prism leader
// asynchronous leader election
// 4 processes
// gxn/dxp 29/01/01

mdp

const int N = 3; // number of processes

//----------------------------------------------------------------------------------------------------------------------------
module process1
	
	// COUNTER
	c1 : [0..3-1];
	
	// STATES
	s1 : [0..4];
	// 0  make choice
	// 1 have not received neighbours choice
	// 2 active
	// 3 inactive
	// 4 leader
	
	// PREFERENCE
	p1 : [0..1];
	
	// VARIABLES FOR SENDING AND RECEIVING
	receive1 : [0..2];
	// not received anything
	// received choice
	// received counter
	sent1 : [0..2];
	// not send anything
	// sent choice
	// sent counter
	
	// pick value
	[pick] (s1=0) -> 0.5 : (s1'=1) & (p1'=0) + 0.5 : (s1'=1) & (p1'=1);

	// send preference
	[p12] (s1=1) & (sent1=0) -> (sent1'=1);
	// receive preference
	// stay active
	[p31] (s1=1) & (receive1=0) & !( (p1=0) & (p3=1) ) -> (s1'=2) & (receive1'=1);
	// become inactive
	[p31] (s1=1) & (receive1=0) & (p1=0) & (p3=1) -> (s1'=3) & (receive1'=1);
	
	// send preference (can now reset preference)
	[p12] (s1=2) & (sent1=0) -> (sent1'=1) & (p1'=0);
	// send counter (already sent preference)
	// not received counter yet
	[c12] (s1=2) & (sent1=1) & (receive1=1) -> (sent1'=2);
	// received counter (pick again)
	[c12] (s1=2) & (sent1=1) & (receive1=2) -> (s1'=0) & (p1'=0) & (c1'=0) & (sent1'=0) & (receive1'=0);
	
	// receive counter and not sent yet (note in this case do not pass it on as will send own counter)
	[c31] (s1=2) & (receive1=1) & (sent1<2) -> (receive1'=2);
	// receive counter and sent counter
	// only active process (decide)
	[c31] (s1=2) & (receive1=1) & (sent1=2) & (c3=N-1) -> (s1'=4) & (p1'=0) & (c1'=0) & (sent1'=0) & (receive1'=0);
	// other active process (pick again)
	[c31] (s1=2) & (receive1=1) & (sent1=2) & (c3<N-1) -> (s1'=0) & (p1'=0) & (c1'=0) & (sent1'=0) & (receive1'=0);
	
	// send preference (must have received preference) and can now reset
	[p12] (s1=3) & (receive1>0) & (sent1=0) -> (sent1'=1) & (p1'=0);
	// send counter (must have received counter first) and can now reset
	[c12] (s1=3) & (receive1=2) & (sent1=1) ->  (s1'=3) & (p1'=0) & (c1'=0) & (sent1'=0) & (receive1'=0);
	
	// receive preference
	[p31] (s1=3) & (receive1=0) -> (p1'=p3) & (receive1'=1);
	// receive counter
	[c31] (s1=3) & (receive1=1) & (c3<N-1) -> (c1'=c3+1) & (receive1'=2);
		
	// done
	[done] (s1=4) -> (s1'=s1);
	// add loop for processes who are inactive
	[done] (s1=3) -> (s1'=s1);

endmodule

//----------------------------------------------------------------------------------------------------------------------------

// construct further stations through renaming
module process2=process1[s1=s2,p1=p2,c1=c2,sent1=sent2,receive1=receive2,p12=p23,p31=p12,c12=c23,c31=c12,p3=p1,c3=c1] endmodule
module process3=process1[s1=s3,p1=p3,c1=c3,sent1=sent3,receive1=receive3,p12=p31,p31=p23,c12=c31,c31=c23,p3=p2,c3=c2] endmodule

//----------------------------------------------------------------------------------------------------------------------------
rewards "rounds"
        [c12] true : 1;
endrewards

//----------------------------------------------------------------------------------------------------------------------------
formula leaders = (s1=4?1:0)+(s2=4?1:0)+(s3=4?1:0);
label "elected" = s1=4|s2=4|s3=4;


In [3]:
leader_model = stormpy.build_model(leader)

In [4]:
print(leader_model)

-------------------------------------------------------------- 
Model type: 	MDP (sparse)
States: 	190
Transitions: 	323
Choices: 	316
Reward Models:  rounds
State Labels: 	3 labels
   * deadlock -> 3 item(s)
   * init -> 1 item(s)
   * elected -> 0 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 



In [5]:
prop = stormpy.parse_properties("Rmax=? [F \"elected\"]", leader)

In [6]:
stormpy_result = stormpy.model_checking(leader_model, prop[0], extract_scheduler=True)

In [7]:
print(stormpy_result.get_values())

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [8]:
scheduler = stormpy_result.scheduler

In [9]:
scheduler_str = []
for state in leader_model.states:
     choice = scheduler.get_choice(state)
     action = choice.get_deterministic_choice()
     scheduler_str.append(f"{state} -> {action}")
print(", ".join(scheduler_str))

0 -> 0, 1 -> 0, 2 -> 0, 3 -> 0, 4 -> 0, 5 -> 0, 6 -> 0, 7 -> 0, 8 -> 0, 9 -> 0, 10 -> 0, 11 -> 0, 12 -> 0, 13 -> 0, 14 -> 0, 15 -> 0, 16 -> 0, 17 -> 0, 18 -> 0, 19 -> 0, 20 -> 0, 21 -> 0, 22 -> 0, 23 -> 0, 24 -> 0, 25 -> 0, 26 -> 0, 27 -> 0, 28 -> 0, 29 -> 0, 30 -> 0, 31 -> 0, 32 -> 0, 33 -> 0, 34 -> 0, 35 -> 0, 36 -> 0, 37 -> 0, 38 -> 0, 39 -> 0, 40 -> 0, 41 -> 0, 42 -> 0, 43 -> 0, 44 -> 0, 45 -> 0, 46 -> 0, 47 -> 0, 48 -> 0, 49 -> 0, 50 -> 0, 51 -> 0, 52 -> 0, 53 -> 0, 54 -> 0, 55 -> 0, 56 -> 0, 57 -> 0, 58 -> 0, 59 -> 0, 60 -> 0, 61 -> 0, 62 -> 0, 63 -> 0, 64 -> 0, 65 -> 0, 66 -> 0, 67 -> 0, 68 -> 0, 69 -> 0, 70 -> 0, 71 -> 0, 72 -> 0, 73 -> 0, 74 -> 0, 75 -> 0, 76 -> 0, 77 -> 0, 78 -> 0, 79 -> 0, 80 -> 0, 81 -> 0, 82 -> 0, 83 -> 0, 84 -> 0, 85 -> 0, 86 -> 0, 87 -> 0, 88 -> 0, 89 -> 0, 90 -> 0, 91 -> 0, 92 -> 0, 93 -> 0, 94 -> 0, 95 -> 0, 96 -> 0, 97 -> 0, 98 -> 0, 99 -> 0, 100 -> 0, 101 -> 0, 102 -> 0, 103 -> 0, 104 -> 0, 105 -> 0, 106 -> 0, 107 -> 0, 108 -> 0, 109 -> 0, 110 -> 0, 

In [13]:
stormvogel_model = map.stormpy_to_stormvogel(leader_model)

stormvogel_result = result.convert_model_checking_result(stormvogel_model, stormpy_result)

print(stormvogel_result)

values: 
 {0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: inf, 6: inf, 7: inf, 8: inf, 9: inf, 10: inf, 11: inf, 12: inf, 13: inf, 14: inf, 15: inf, 16: inf, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: inf, 25: inf, 26: inf, 27: inf, 28: inf, 29: inf, 30: inf, 31: inf, 32: inf, 33: inf, 34: inf, 35: inf, 36: inf, 37: inf, 38: inf, 39: inf, 40: inf, 41: inf, 42: inf, 43: inf, 44: inf, 45: inf, 46: inf, 47: inf, 48: inf, 49: inf, 50: inf, 51: inf, 52: inf, 53: inf, 54: inf, 55: inf, 56: inf, 57: inf, 58: inf, 59: inf, 60: inf, 61: inf, 62: inf, 63: inf, 64: inf, 65: inf, 66: inf, 67: inf, 68: inf, 69: inf, 70: inf, 71: inf, 72: inf, 73: inf, 74: inf, 75: inf, 76: inf, 77: inf, 78: inf, 79: inf, 80: inf, 81: inf, 82: inf, 83: inf, 84: inf, 85: inf, 86: inf, 87: inf, 88: inf, 89: inf, 90: inf, 91: inf, 92: inf, 93: inf, 94: inf, 95: inf, 96: inf, 97: inf, 98: inf, 99: inf, 100: inf, 101: inf, 102: inf, 103: inf, 104: inf, 105: inf, 106: inf, 107: inf, 108: inf, 109: inf,

In [12]:
visualization.show(stormvogel_model, stormvogel_result)